In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("spotify_millsongdata.csv")

In [5]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [7]:
df.shape

(57650, 4)

In [8]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [9]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [10]:
df.head(10)

,artist,song,text
0,Doobie Brothers,Closer Every Day,Patrick Simmons \r\nI just looked in the mirr...
1,Usher,Let It Burn,"[Intro] \r\nGirl, understand why \r\nSee it'..."
2,Grateful Dead,Aiko Aiko,My spy boy saw you spy boy sittin by the bi-yo...
3,HIM,Love's Requiem,Confusion writhes around our hearts impatientl...
4,Ween,Loving U Thru It All,"Loving you through it all, thick and thin, \r..."
5,Nick Cave,Knockin' On Joe,"These chains of sorrow, they are heavy, it is ..."
6,Ray Charles,Sticks And Stones,People talkin' tryin' to break us up \r\nWhy ...
7,Don Henley,Inside Out,The biggest lie you ever told \r\nYour deepes...
8,Electric Light Orchestra,Time After Time,"Time after time, time after time \r\nThe skie..."
9,Incognito,This Thing Called Love,Here we are \r\nTrying to find a meaning for ...


In [11]:
df['text'][0]

"Patrick Simmons  \r\nI just looked in the mirror, was that me  \r\nI'm older now than I used to be  \r\nThe sky above me, lord, it's turnin' red  \r\nThere's a hard rain fallin' 'round my head  \r\nOh, lord, fallin' 'round my head  \r\nOh, lord, fallin' 'round my head  \r\nOh, lord, fallin' 'round my head  \r\nOh, lord, fallin' 'round my head  \r\n  \r\nThe road I'm standin' on, it twists and turns  \r\nThe soles of my feet, lord, they're 'bout to burn  \r\nThe reaper beckons, and he points the way  \r\nOh, lord, I'm gettin' closer every day  \r\nOh, lord, closer every day  \r\nOh, lord, closer every day  \r\nOh, lord, closer every day  \r\nOh, lord, closer every day\r\n\r\n"

In [12]:
# df = df.sample(5000)

In [13]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [17]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [16]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
 
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [22]:
similarity[0]

array([1.00000000e+00, 7.74224264e-02, 1.01487723e-02, ...,
       3.97369880e-04, 1.18181247e-02, 2.06858697e-02])

In [23]:
df[df['song'] == 'Crying Over You']

,artist,song,text


In [24]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [26]:
recommendation('Closer Every Day')

['Oh My Lord',
 'Lord, In Your Love',
 'Even Me',
 'Thank You Lord',
 "Fallin' Apart At The Seams (Live)",
 'Sinnerman',
 'Say You Love Me',
 'Earth Blues',
 'Consuming Fire',
 "I've Got A Feelin' I'm Fallin'",
 'Let Everything That Has Breath',
 'Away In A Manger',
 'Away In The Manger',
 'Away In A Manger',
 'Million Dollar Bill',
 'Away In A Manger',
 'Away In A Manger',
 'Honda Accord',
 'Jesus',
 "Travelin' Prayer"]

In [27]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))